# Проект по определению возможного оттока клиентов

Из банка, заказчика исследования, стали уходить клиенты. 

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Нам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Необходимо построить модель с предельно большим значением *F1*-меры. Минимально необходимое значение метрики равно 0.59.

Необходимо дополнительно измерить *AUC-ROC* и сравнить её значение с *F1*-мерой.

Признаки данных:
```

RowNumber — индекс строки в данных
CustomerId — уникальный идентификатор клиента
Surname — фамилия
CreditScore — кредитный рейтинг
Geography — страна проживания
Gender — пол
Age — возраст
Tenure — сколько лет человек является клиентом банка
Balance — баланс на счёте
NumOfProducts — количество продуктов банка, используемых клиентом
HasCrCard — наличие кредитной карты
IsActiveMember — активность клиента
EstimatedSalary — предполагаемая зарплата

```

Целевой признак

```
Exited — факт ухода клиента

```

## Подготовка данных

Импортируем необходимые библиотеки:

In [3]:
#standard
import pandas as pd
import numpy as np

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import f1_score, roc_auc_score, recall_score 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.dummy import DummyClassifier

#imblearn
from imblearn.over_sampling import SMOTE

Загрузим рабочий датасет:

In [2]:
data = pd.read_csv('data')

Изучим:

In [5]:
data.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


В данных есть информация о 10000 клиентов, сразу видно, что есть пропуски в графе `Tenure`. Так же в рамках нашего исследования нам не понадобятся некоторые колонки - индекс строки, уникальный идентификатор клиента и его фамилия. Эти колонки можно удалить. А вот с кредитным рейтингом, страной проживания, полом, возрастом, тем сколько клиент работает с банком, его баланс на счете, количество продуктов, наличие кредитной карты, его активность и зарплата это то, с чем мы будем работать.

Проведем проверку на дубликаты:

In [7]:
data.duplicated().sum()

0

Дубликатов нет.

Как отметили раньше в данных пропуски в столбце `Tenure`, удалим их из наших данных:

In [8]:
data = data.dropna().reset_index(drop=True)

Теперь приведем названия столбцов в соответствии с принятым стилем:

In [9]:
data = data.rename(columns={'RowNumber':'row_number', 'CustomerId':'customer_id', 'CreditScore':'credit_score', 'NumOfProducts':'num_of_products', 'HasCrCard':'has_cr_card', 'IsActiveMember':'is_active_member', 'EstimatedSalary':'estimated_salary', })

In [10]:
data.columns= data.columns.str.lower()

Теперь удалим колонки, которые не подойдут для дальнешего исследования:

In [11]:
data.drop(['row_number', 'customer_id', 'surname'], axis=1, inplace=True)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9091 entries, 0 to 9090
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   credit_score      9091 non-null   int64  
 1   geography         9091 non-null   object 
 2   gender            9091 non-null   object 
 3   age               9091 non-null   int64  
 4   tenure            9091 non-null   float64
 5   balance           9091 non-null   float64
 6   num_of_products   9091 non-null   int64  
 7   has_cr_card       9091 non-null   int64  
 8   is_active_member  9091 non-null   int64  
 9   estimated_salary  9091 non-null   float64
 10  exited            9091 non-null   int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 781.4+ KB


Теперь, когда мы избавились от пропусков, подтвердили отсутствие дубликатов и убали ненужные столбцы и стилевые нарушения можно начать наше исследование.

## Исследование задачи

Для того, чтобы обучить модель и проверить ее работу понадобится разбить данные на три выборки: учебную, валидационную и тестовую. Но перед этим разобьем выборку на признаки и целевые признаки. В целевые признаки `target` возьмем столбец с информацией о том ушел клиент или нет, в обычные признаки `features` пойдут все остальные столбцы:

In [13]:
features = data.drop(['exited'], axis=1)
target = data['exited']

Для разбития данных на выборки используем `train_test_split`. Сначала разобьем на тренировочную выборку `train` и выборку для проверки `check` в соотношении 60% и 40%, потом выборку для проверки `check` разобьем пополам, чтобы получилось по 20% от всей выборки на валидационную `valid` и тестовую `test` выборки:

In [14]:
features_train, features_check, target_train, target_check, = train_test_split(
    features, target, test_size=0.40, random_state=12345)

In [15]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_check, target_check, test_size=0.50, random_state=12345)

Проверим, чтобы количественно выборки соответствовали необходимым параметрам:

In [16]:
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(5454, 10)
(5454,)
(1818, 10)
(1818,)
(1819, 10)
(1819,)


Выборки подготовленны.

Следующим шагом будет преобразование категориальных признаков в численные, для этого используем возможности `OneHotEncoder`. Для начала определим, с какими столбцами будем работать:

In [17]:
categorical = ['geography', 'gender']

Теперь обучим функцию на нашей тренировочной выборке.

In [18]:
ohe = OneHotEncoder(handle_unknown='ignore',sparse_output=False, drop='first')

ohe.fit(features_train[categorical])

OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)

И перекодируем каждую нашу выборку с помощью `OneHotEncoder`:

Тренировочную:

In [19]:
features_train_encoded = features_train.copy()

features_train_encoded[ohe.get_feature_names_out()] = ohe.transform(features_train[categorical])

features_train_encoded = features_train_encoded.drop(categorical, axis=1)

Валидационную:

In [20]:
features_valid_encoded = features_valid.copy()

features_valid_encoded[ohe.get_feature_names_out()] = ohe.transform(features_valid[categorical])

features_valid_encoded = features_valid_encoded.drop(categorical, axis=1)

И тестовую:

In [21]:
features_test_encoded = features_test.copy()

features_test_encoded[ohe.get_feature_names_out()] = ohe.transform(features_test[categorical])

features_test_encoded = features_test_encoded.drop(categorical, axis=1)

Проверим как теперь выглядят столбцы на примере тренировочной выборки:

In [22]:
features_train_encoded.head(3)

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
8483,727,28,2.0,110997.76,1,1,0,101433.76,0.0,0.0,0.0
3431,537,26,7.0,106397.75,1,0,0,103563.23,0.0,0.0,1.0
6770,610,40,9.0,0.00,1,1,1,149602.54,0.0,0.0,1.0


Для дальнейшей работы масштабируем наши выборки. Сначала выберем численные признаки для масштабирования:

In [23]:
numeric = ['credit_score','age','tenure','balance','num_of_products','has_cr_card','is_active_member','estimated_salary']

Теперь создадим объект масштабирования и настроим его на обучающей выборке:

In [24]:
scaler = StandardScaler()
scaler.fit(features_train_encoded[numeric]) 
features_train_encoded[numeric] = scaler.transform(features_train_encoded[numeric])
features_valid_encoded[numeric] = scaler.transform(features_valid_encoded[numeric])
features_test_encoded[numeric] = scaler.transform(features_test_encoded[numeric])

Проверим получилось ли масштабировать по учебной выборке:

In [25]:
features_train_encoded.head(3)

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
8483,0.809075,-1.039327,-1.025995,0.554904,-0.908179,0.663468,-1.024127,0.019508,0.0,0.0,0.0
3431,-1.152518,-1.227561,0.696524,0.480609,-0.908179,-1.507231,-1.024127,0.056167,0.0,0.0,1.0
6770,-0.398853,0.090079,1.385532,-1.237830,-0.908179,0.663468,0.976442,0.848738,0.0,0.0,1.0


Теперь начнем изучать какие модели лучше себя поведут:

Обучим и используем три модели:
- Модель дерево решений (Decision Tree Classifier)
- Модель случайного леса (Random Fores tClassifier)
- Модель логистической регрессии (Logistic Regression)

Начнем с модели дерева решений `model_dtc`. Установим какой параметр максимальной глубины дерева `depth` (от 1 до 11) даст модель с  лучшим показателем *F1* при сравнении с ответами валидационной выборки.

In [26]:
best_result_dtc = 0
best_depth_dtc = 0

for depth in range(1, 11):
    model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_dtc.fit(features_train_encoded, target_train)
    predictions_valid_dtc = model_dtc.predict(features_valid_encoded)
    result_valid_dtc = f1_score(target_valid, predictions_valid_dtc)
    if result_valid_dtc > best_result_dtc:
        best_result_dtc = result_valid_dtc
        best_depth_dtc = depth
        
    
print(' F1-мера наилучшей модели дерева решений на валидационной выборке:', best_result_dtc, 
'\n При максимальной глубине дерева:', best_depth_dtc)

 F1-мера наилучшей модели дерева решений на валидационной выборке: 0.5764331210191083 
 При максимальной глубине дерева: 7


Теперь очередь модели случайного леса `model_rfc`. Установим какой параметр количества деревьев `est` и какая глубина дерева `depth` даст модель с лучшим показателем *F1* при сравнении с ответами валидационной выборки.

In [27]:
best_result_rfc = 0
best_est_rfc = 0
best_depth_rfc = 0

for est in range(1, 16):
    for depth in range(10, 26):
        model_rfc = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_rfc.fit(features_train_encoded, target_train)
        predictions_valid_rfc = model_rfc.predict(features_valid_encoded)
        result_valid_rfc = f1_score(target_valid, predictions_valid_rfc)
        if result_valid_rfc > best_result_rfc:
            best_result_rfc = result_valid_rfc
            best_est_rfc = est
            best_depth_rfc = depth
            

        
print(' F1-мера наилучшей модели случайного леса на валидационной выборке:', best_result_rfc,
      '\n При количестве деревьев:', best_est_rfc, 'При глубине деревьев', best_depth_rfc)

 F1-мера наилучшей модели случайного леса на валидационной выборке: 0.60625 
 При количестве деревьев: 13 При глубине деревьев 24


Теперь проверим модель логистической регрессии `model_lr`. Установим какой параметр итерации `itr` (от 100 до 1000, с шагом в 10) даст модель с лучшим показателем *F1*  при сравнении с ответами валидационной выборки.

In [28]:
best_result_lr = 0
best_itr_lr = 0

for itr in range(100, 501, 10):
    model_lr = LogisticRegression(random_state=12345, solver='liblinear', max_iter=itr)
    model_lr.fit(features_train_encoded, target_train)
    predictions_valid_lr = model_lr.predict(features_valid_encoded)
    result_valid_lr = f1_score(target_valid, predictions_valid_lr)
    if result_valid_lr > best_result_lr:
        best_result_lr = result_valid_lr
        best_itr_lr = itr

print(' F1-мера наилучшей модели логистической регрессии на валидационной выборке:', best_result_lr,
      '\n При количестве итераций:', best_itr_lr)

 F1-мера наилучшей модели логистической регрессии на валидационной выборке: 0.3033932135728543 
 При количестве итераций: 100


Мы разбили наши данные на три выборки, необходимые для работы, а после поменяли категориальные признаки в численные. Затем после исследования трех различных моделей мы установиил, что лучше всего себя показала модель случайного леса с количеством деревьев 39 и глубиной 24. Так как эта модель лучшая, то после того, как уберем дисбаланс в учебной выборке, то мы снова ее обучим и проверим ее качество.

## Борьба с дисбалансом

Проверим есть ли дисбаланс классов:

In [29]:
np.unique(target_train, return_counts = True)

(array([0, 1]), array([4328, 1126]))

Дисбаланс присутствует, отрицательных значений больше в 4 раза. С помощью функции SMOTE поправим дисбаланс классов в выборке:

In [32]:
oversample = SMOTE(random_state=12345)
features_train_u, target_train_u = oversample.fit_resample(features_train_encoded, target_train)

Теперь у нас есть сбалансированные выборки для дальнейшего исследования:

Снова начнем исследование с модели дерева решений `model_dtc`:

In [33]:
best_result_dtc = 0
best_depth_dtc = 0

for depth in range(1, 11):
    model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_dtc.fit(features_train_u, target_train_u)
    predictions_valid_dtc = model_dtc.predict(features_valid_encoded)
    result_valid_dtc = f1_score(target_valid, predictions_valid_dtc)
    if result_valid_dtc > best_result_dtc:
        best_result_dtc = result_valid_dtc
        best_depth_dtc = depth
        
    
print(' F1-мера наилучшей модели дерева решений на валидационной выборке:', best_result_dtc, 
'\n При максимальной глубине дерева:', best_depth_dtc)

 F1-мера наилучшей модели дерева решений на валидационной выборке: 0.5956964892412231 
 При максимальной глубине дерева: 8


На сбалансированных выборках модель показывает результат получше.

Теперь проверим модель логистической регрессии `model_lr`:

In [34]:
best_result_lr = 0
best_itr_lr = 0

for itr in range(100, 501, 10):
    model_lr = LogisticRegression(random_state=12345, solver='liblinear', max_iter=itr)
    model_lr.fit(features_train_u, target_train_u)
    predictions_valid_lr = model_lr.predict(features_valid_encoded)
    result_valid_lr = f1_score(target_valid, predictions_valid_lr)
    if result_valid_lr > best_result_lr:
        best_result_lr = result_valid_lr
        best_itr_lr = itr

print(' F1-мера наилучшей модели логистической регрессии на валидационной выборке:', best_result_lr,
      '\n При количестве итераций:', best_itr_lr)

 F1-мера наилучшей модели логистической регрессии на валидационной выборке: 0.5079962370649106 
 При количестве итераций: 100


Модель логистической регресии на сбалансированных выборках показывает гораздо более хороший результат

Теперь на этих сбалансированных выборках обучим модель случайного леса `model_rfc`, показавшую до этого самый лучший результат, чтобы определить какие гиперпараметры задать модели:

In [35]:
best_result_rfc = 0
best_est_rfc = 0
best_depth_rfc = 0

for est in range(30, 41):
    for depth in range(1, 16):
        model_rfc = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_rfc.fit(features_train_u, target_train_u)
        predictions_valid_rfc = model_rfc.predict(features_valid_encoded)
        result_valid_rfc = f1_score(target_valid, predictions_valid_rfc)
        if result_valid_rfc > best_result_rfc:
            best_result_rfc = result_valid_rfc
            best_est_rfc = est
            best_depth_rfc = depth
            

        
print(' F1-мера наилучшей модели случайного леса на валидационной выборке:', best_result_rfc,
      '\n При количестве деревьев:', best_est_rfc, 'При глубине деревьев', best_depth_rfc)

 F1-мера наилучшей модели случайного леса на валидационной выборке: 0.655483870967742 
 При количестве деревьев: 36 При глубине деревьев 11


После проведения балансировки модель случайного леса снова показывает себя лучше в предсказаниях.

Сохраним полученную лучшую модель с полученными гиперпараметрами для проверки ее на тестовой выборке.

In [36]:
best_model = RandomForestClassifier(random_state=12345, n_estimators=36, max_depth=11)
best_model.fit(features_train_u, target_train_u)

RandomForestClassifier(max_depth=11, n_estimators=36, random_state=12345)

После корректировки дисбаланса класса с помощью функции SMOTE получилось улучшить показатель `F1` наших моделей. Более хороший показали и модель дерева решений и модель логической регрессии, однако как и при иссследовании до корректировки дисбаланса лучше других себя показала модель случайного леса. Мы переобучили ее и подобрали новые гиперпараметры (количество деревьев 36, глубина 11), теперь пришло время опробовать ее на тестовой выборке.

## Тестирование модели

Протестируем нашу обученную модель на тестовой выборке:

In [37]:
predictions_test = best_model.predict(features_test_encoded)
f1_score(target_test, predictions_test)

0.601078167115903

Модель показала себя хуже, чем на валидационной выборке, но F1-мера превышает необходимый уровень в 0.59.

Для проверки на адекватность создадим dummy-модель, обучим ее на подготовленных учебных выборках и узнаем ее F1-меру для тестовой выборки:

In [38]:
dummy_model = DummyClassifier(strategy='constant', constant=1)
dummy_model.fit(features_train_u, target_train_u)
f1_score(dummy_model.predict(features_test_encoded), target_test)

0.3235023041474654

Проверку на адекватность наша модель прошла.

Теперь изучим модель с помощью допольнительной оценки AUC-ROC:

In [39]:
probabilities_test = best_model.predict_proba(features_test_encoded)
probabilities_one_valid = probabilities_test[:, 1]

print(roc_auc_score(target_test, probabilities_one_valid))

0.855975919327418


Здесь модель тоже проходит проверку, показатель выше критического 0.5.

Так как для нас важно обнаружнить тех клиентов, которые планируют уйти, то проверим нашу модель по метрике полноты:

In [40]:
recall_score(target_test, predictions_test)

0.6353276353276354

**Вывод по проекту:**

Мы проверили работу нескольких моделей и выбрали из них наилучшую модель, это модель случайного леса с гиперпараметрами: количество деревьев 36, глубина 11.

На тестово выборке она показала результат F1-меры равный 0.603, отвечающий нашим требованиям и гораздо более высокий чем у dummy-модели 0.323.

Показатель ROC AUC модели составляет 0,856, что говорит о том, что из предсказанных моделью клиентов готовых рассторгнуть договор 85% действительно собираются это сделать.

Так же проверен на тестовой выборке показатель полноты, он равен 0.635, это означает, что из тех клиентов, которые планируют уходить, модель обнаруживает 63%.